In [13]:
import os

import pandas as pd
from dotenv import load_dotenv

openai_key = os.getenv('MEENALI_OPENAI_API_KEY')

In [14]:
from langchain_community.llms import Ollama
# from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
# from langchain_openai.embeddings import OpenAIEmbeddings

MODEL = "llama3"
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

"Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(wait for it...)\n\nBecause it was two-tired!\n\nHope that made you smile! Do you want to hear another one?"

In [15]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

"Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!\n\nHope that made you laugh!"

In [16]:
from langchain.prompts import PromptTemplate

template = """
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided. If you don't know the answer say You don't know

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nYou are a Q&A assistant. Your goal is to answer questions as\naccurately as possible based on the instructions and context provided.\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [17]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Pavan Gupta", "question": "What's your name'?"})

'According to the context, my name is Pavan Gupta!'

In [18]:
# from langchain_community.document_loaders import PyPDFLoader

# loader = PyPDFLoader("data/1700394406_Guideline-for-Insurance-Policies-and-Proceedings_2023.pdf")
# pages = loader.load_and_split()
# pages

from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("data/")
pages = loader.load()
pages


[Document(metadata={'source': 'data/1716531735_Rewards &amp_ Recognition Policy__2024_V2.pdf'}, page_content="Standard Policy Document: Rewards and Recognition Policy\n\nRewards and Recognition Policy\n\nGAVS Technologies Private Limited\n\n(Great Software Laboratory Private Limited is merged into GAVS Technologies Private Limited vide National Company Law Tribunal (NCLT) order)\n\nRegistered Office: New No. 13 (Old No. 11), Rajiv Gandhi Salai (OMR) Sholinganallur, Kancheepuram, Chennai - 600 119, Tamil Nadu. Tel: +91 44 6669 4287 Pune Office: 8th Floor (A and B), Amar Arma Genesis, Baner Road, Pune – 411 045 Tel: +91 020 6310 6500 / 020 6310 6600 CIN No: U72200TN2007PTC062974 | www.gavstech.com , www.gslab.com\n\n1 Copyright@2024 GS Lab and/ or GAVS as applicable. All rights reserved. Private & Confidential\n\nStandard Policy Document: Rewards and Recognition Policy\n\nDocument History\n\nName\n\nDesignation\n\nPrepared by\n\nPuvitha Raju\n\nReviewed by\n\nPoonam\n\nApproved by\n\nSan

In [19]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [20]:
retriever = vectorstore.as_retriever()
retriever.invoke("machine learning")

[Document(metadata={'source': 'data/PolarisIO-case-study.pdf'}, page_content='Customer details (Name needs to be masked)\n\nPolaris I/O stands as an all-encompassing enterprise platform designed for customer interaction and obtaining insights to lead the organization towards scaling their existing key accounts. It seamlessly integrates transformation services, industry expertise, and cutting-edge technology, empowering B2B enterprises to safeguard, retain, and expand their major accounts. PolarisIO’s unique "wellness programs" adopt an external perspective to enhance your business vitality. By leveraging insights from executive initiatives to enhance relationships, buyers, they strategically coordinate commercial optimize growth, and extend impact beyond our initial involvement.\n\nKey account focus\n\nPolaris I/O is crafted to underpin the systematic expansion and the enduring commercial well-being of an enterprise. Effectively scaling go-to-customer strategies is crucial for the syst

In [21]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [28]:
questions = [
    "When will Leaves become valid?",
    "Annual variable policy?",
    "Who is contact person for HR policies?"
    "What is maternity leave policy guidelines?",
    # "Get me the Pune and Vadodara holiday list",
    # "What is Group Personal Accident Policy?",
    # "PolarisIO Applications?"
    # "What is open heart procedure?"
    # "What are the Policies in effect?",
    # "How much is the night shift allowns?",
    # "High Level Overview of the insurrance Process?",
    # "What is Group Term Life Insurance Policy",
    # "How many coding assignments are there in the program?",
    # "Is there a program certificate upon completion?",
    # "What programming language will be used in the program?",
    # "How much does the program cost?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: When will Leaves become valid?
Answer: Unfortunately, the provided document does not mention when leaves (vacation days) will become valid. The document only lists published holidays for various locations (India, United Kingdom, United Arab Emirates, Oman, and Bahrain). It does not provide information on when leave days can be taken or how to request them. If you have a specific query regarding leaves or employee benefits, I suggest contacting the relevant HR department or supervisor for more information.

Question: Annual variable policy?
Answer: There is no information provided about an "Annual Variable Policy" in this document. The document appears to be an employee onboarding process outline, detailing the steps involved in bringing a new employee up to speed and integrating them into the organization. It covers topics such as creating personnel files, conducting orientation, and issuing access cards and email IDs. There is no mention of an "Annual Variable Policy".

Ques

In [23]:
# import openai

# def check_openai_api_key(api_key):
#     client = openai.OpenAI(api_key=api_key)
#     try:
#         client.models.list()
#     except openai.AuthenticationError:
#         return False
#     else:
#         return True

# if check_openai_api_key(openai_key):
#     print("Valid OpenAI API key.")
# else:
#     print("Invalid OpenAI API key.")

In [24]:
# from openai import OpenAI

# client = OpenAI(api_key=openai_key)

# completion = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=[
#         {
#             "role": "system",
#             "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair.",
#         },
#         {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."},
#     ],
# )

# print(completion.choices[0].message)